In [1]:
import os
import re
import pandas as pd
import numpy as np
from dplython import (DplyFrame, X, diamonds, select, sift, sample_n, sample_frac, arrange, mutate, group_by, summarize)

import plotly.plotly
import plotly.graph_objs as go
import pandas as pd

import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
from plotly import tools

from util_functions import *

%config IPCompleter.greedy=True

in_data_path = '/Users/genie/data/oflc/kaggle/h1b_data_fy2011_fy2018_20190309.csv'
cities_data_path = '/Users/genie/data/oflc/us_cities.csv'
# cities_data_path = '/Users/genie/data/oflc/uscitiesv1.3.csv'
coli_data_path = '/Users/genie/data/numbeo/cost-of-living/cost-of-living-2018.csv'
metro_data_path = '/Users/genie/data/oflc/metro_areas_by_county.csv'

out_dir = '/Users/genie/dev/projects/github/h1b_data_analysis/out'
# out_dir = '/Users/genie/dev/projects/github/genielab_github_io/assets/custom/h1b_data_analysis/'
os.chdir(out_dir)

include_plotlyjs=False

/anaconda3/envs/dev2/lib/python3.7/site-packages/socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
/anaconda3/envs/dev2/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__



### Load Data

In [2]:
df = pd.read_csv(in_data_path,quotechar='"',na_values='',encoding = "ISO-8859-1")
cities_df = pd.read_csv(cities_data_path,quotechar='"',na_values='',encoding = "ISO-8859-1")
metro_df = pd.read_csv(metro_data_path,quotechar='"',na_values='',encoding = "ISO-8859-1")

/anaconda3/envs/dev2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning:

Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.



### Data Cleansing/New Variables

In [3]:
df = df.drop(['case_submitted','decision_date','full_time_position'],axis=1)

# we need to derive all these variables again as they are not accurate, should be dropped in next kaggle dataset version
df = df.drop(['county_fips','county_name','metro','lat','lng'], axis=1)

df['state'] = df.apply(lambda x: x['work_state'] if pd.notnull(x['work_state']) else x['emp_state'] ,axis=1)
df['city'] = df.apply(derive_city_variable, axis=1)

df.city[(pd.notnull(df.city)) & (df.city.str.match("Newyork")==True)] = "New York"
df.city[(pd.notnull(df.city)) & (df.city.str.match("New York City")==True)] = "New York"
df.city[(pd.notnull(df.city)) & (df.city.str.match("Santa Clara")==True)] = "Santa Clara"
df.city[(pd.notnull(df.city)) & (df.city.str.match("San Fransisco")==True)] = "San Francisco"

df['city_state'] = df.apply(lambda x: ','.join([x['city'].title(),x['state']]) if pd.notnull(x['city']) and pd.notnull(x['state']) else None,axis=1)

df = df.drop(['emp_city','emp_state','emp_zip','work_city','work_state','work_zip'], axis=1)

df = pd.merge(df, cities_df[['city_name','state_id','county_fips','county_name']], how='left', left_on=['city','state'], right_on=['city_name','state_id'])
df = df.drop(['city_name','state_id'],axis=1)

df = pd.merge(df, metro_df[['county_fips','metro_name']], how='left', left_on='county_fips', right_on='county_fips')
df.rename(columns={'metro_name':'metro'}, inplace=True)


### Demand Analysis

In [4]:
# fiscal year counts - overall
df1 = DplyFrame(df) >> sift(X.case_status.notnull()) >> group_by(X.fiscal_year) >> summarize(fy_total=X.case_status.count())
df2 = DplyFrame(df) >> sift(X.case_status=="C") >> group_by(X.fiscal_year) >> summarize(fy_certified=X.case_status.count())
oflc_fy_counts = pd.merge(df1,df2,on='fiscal_year')

del df1
del df2

### Geographic Analysis

In [5]:
# chart
# fiscal year counts - by state

df1 = DplyFrame(df) >> sift(X.case_status.notnull(),X.state.notnull()) >> group_by(X.fiscal_year, X.state) >> summarize(fy_state_total=X.case_status.count())
df2 = DplyFrame(df) >> sift(X.case_status=="C",X.state.notnull()) >> group_by(X.fiscal_year, X.state) >> summarize(fy_certified_state=X.case_status.count())
df1 = pd.merge(df1,df2,on=['fiscal_year','state'])
del df2

oflc_fy_state_counts = pd.merge(df1, oflc_fy_counts, how='left', on='fiscal_year')
del df1

oflc_fy_state_counts['fy_state_total_share_pct'] =  oflc_fy_state_counts.apply(lambda x: round(x['fy_state_total']/x['fy_total'] * 100,2), axis=1)
oflc_fy_state_counts['fy_state_certified_share_pct'] =  oflc_fy_state_counts.apply(lambda x: round(x['fy_certified_state']/x['fy_certified'] * 100,2), axis=1)
oflc_fy_state_counts = oflc_fy_state_counts.drop(['fy_certified_state','fy_total','fy_certified'],axis=1)

zmin = oflc_fy_state_counts['fy_state_total'].min()
zmax = oflc_fy_state_counts['fy_state_total'].max()

scl = [[0.0, '#ffffff'],[0.2, '#ff9999'],[0.4, '#ff4d4d'], [0.6, '#ff1a1a'],[0.8, '#cc0000'],[1.0, '#4d0000']] # reds

# create data for slider
data_slider = []
for year in oflc_fy_state_counts.fiscal_year.unique():
    df_temp = oflc_fy_state_counts[(oflc_fy_state_counts['fiscal_year']== year)]
    
    for col in df_temp.columns:
        df_temp[col] = df_temp[col].astype(str)
    
    df_temp['text'] = 'FY' + df_temp['fiscal_year'] + ' Share (%) ' + df_temp['fy_state_total_share_pct'] 
    
    data_temp = dict(
        type='choropleth',
        colorscale = scl,
        zmin=zmin, 
        zmax=zmax,
        autocolorscale = False,
        locations = df_temp['state'],
        z = df_temp['fy_state_total'].astype(float),
        locationmode = 'USA-states',
        text = df_temp['text'],
        marker = dict(line = dict(color = 'rgb(255,255,255)',width = 2)),
        colorbar = dict(title = "Total applications"))
    
    data_slider.append(data_temp)

# create steps
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 2011)) # label to be displayed for each step (year)
    step['args'][1][i] = True
    steps.append(step)


sliders = [dict(active=0, pad={"t": 1}, steps=steps)] 

layout = go.Layout(title=go.layout.Title(text='2011-2018 Statewide H-1B Application Volume<br>(Hover for breakdown)'),
                   geo=go.layout.Geo(scope='usa',projection=go.layout.geo.Projection(type='albers usa'),
                                     showlakes=True,lakecolor = 'rgb(255, 255, 255)'),
                   sliders=sliders)

fig = go.Figure(data = data_slider, layout = layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_statewide_dist.html', validate=True)

del oflc_fy_state_counts

In [6]:
# chart
# fiscal year counts by city

df1 = DplyFrame(df) >> sift(X.case_status.notnull(),X.city_state.notnull()) >> group_by(X.fiscal_year, X.city_state) >> summarize(fy_city_total=X.case_status.count())

oflc_fy_city_counts = pd.merge(df1, oflc_fy_counts, how='left', on='fiscal_year')
del df1

oflc_fy_city_counts['fy_city_total_share_pct'] =  oflc_fy_city_counts.apply(lambda x: round(x['fy_city_total']/x['fy_total'] * 100,2), axis=1)
oflc_fy_city_counts = DplyFrame(oflc_fy_city_counts) >> sift(X.fy_city_total>50) >> select(X.fiscal_year,X.city_state,X.fy_city_total,X.fy_city_total_share_pct)
oflc_fy_city_counts['key'] = oflc_fy_city_counts['city_state'].apply(lambda x: x.upper() if x is not None else None) 

cities_df['key'] = cities_df.apply(lambda x: x['city_name'] + ',' + x['state_id'], axis=1)

oflc_fy_city_counts = pd.merge(oflc_fy_city_counts, cities_df, how='left', on='key')
del cities_df

oflc_fy_city_counts = oflc_fy_city_counts.drop(['key'],axis=1)
oflc_fy_city_counts = oflc_fy_city_counts.dropna()

x_min = float(oflc_fy_city_counts['fy_city_total'].min())
x_max = float(oflc_fy_city_counts['fy_city_total'].max())
norm_x = 10.0
norm_y = 50.0

oflc_fy_city_counts['size'] = oflc_fy_city_counts.apply(lambda x : round(norm_x + (norm_y-norm_x)*(x['fy_city_total']-x_min)/(x_max-x_min),0), axis=1)

scl = [[0.0,'#ffe5e5'],[0.1, '#ffcccc'], [0.2, '#ffb2b2'], [0.3, '#ff9999'],[0.4, '#ff7f7f'], [0.5, '#ff6666'], [0.6, '#ff4c4c'], [0.7, '#ff3232'], 
       [0.8, '#ff1919'], [0.9, '#ff0000'], [1.0, '#cc0000']] # reds

cmax=oflc_fy_city_counts['fy_city_total'].max()

# create data for slider
data_slider = []
for year in oflc_fy_city_counts.fiscal_year.unique():
    df_temp = oflc_fy_city_counts[(oflc_fy_city_counts['fiscal_year']== year)]
    
    for col in df_temp.columns:
        df_temp[col] = df_temp[col].astype(str)
    
    df_temp['text'] = df_temp['city_state'] + '<br>FY' + df_temp['fiscal_year'] + ' Total Applications:' + df_temp['fy_city_total'] + '<br>Share (%) ' + df_temp['fy_city_total_share_pct'] 
    
    data_temp = go.Scattergeo(
#         type='scattergeo',
        locationmode = 'USA-states',
        lon = df_temp['lng'].astype(float),
        lat = df_temp['lat'].astype(float),
        text = df_temp['text'],
        mode = 'markers',
        marker = dict( 
            size = df_temp['size'].astype(float), 
            opacity = 0.8,
            symbol = 'circle',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            cmin = 0,
            cmax = cmax,
            colorscale=scl,
            autocolorscale = False,
#             reversescale = True,
            color = df_temp['fy_city_total'].astype(float),
            colorbar=dict(
                title="H-1B distributions by city"
            )
        ))
    
    data_slider.append(data_temp)

# create steps
steps = []
for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='Year {}'.format(i + 2011)) # label to be displayed for each step (year)
    step['args'][1][i] = True
    steps.append(step)


sliders = [dict(active=0, pad={"t": 1}, steps=steps)] 

layout = dict(
        title = '2011-2018 H-1B distributions by city', 
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            showlakes=True,lakecolor = 'rgb(255, 255, 255)',
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5        
        ),
        sliders=sliders
    )

fig = go.Figure(data = data_slider, layout = layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_city_dist.html', validate=True)

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_city_dist.html'

### Salary Analysis in Selected Cities

In [7]:
selected_metros = ['Seattle', 'SF Bay Area', 'Dallas', 'Miami','Atlanta', 'Detroit-Ann Arbor', 'Boston', 'Los Angeles', 'New York-Newark',
       'Charlotte', 'Chicago', 'Tampa', 'Raleigh-Durham', 'Austin', 'Pittsburgh', 'San Diego','Washington DC', 'Philadelphia', 'Milwaukee', 
       'Phoenix', 'Houston','Denver', 'Minneapolis-St.Paul', 'St. Louis', 'Nashville', 'Orlando', 'Kansas City', 'Las Vegas','Sacremento', 
        'Richmond', 'Portland', 'Tulsa', 'Anchorage', 'Honolulu']

selected_metros = sorted(selected_metros)

df_certified_2018 = df[(df.fiscal_year==2018) & (df.case_status=="C") & (pd.notnull(df.metro)) & (df.metro.isin(selected_metros))]
df_certified_2018['avg_annual_wage'] = df_certified_2018.apply(calc_emp_avg_annual_wage, axis=1)

# oflc_2018_selected_metros = df[(df.fiscal_year==2018) & (df.case_status=="C") & (df.metro.isin(selected_metros))]
# oflc_2018_selected_metros['avg_annual_wage'] = oflc_2018_selected_metros.apply(calc_emp_avg_annual_wage, axis=1)

In [8]:
# federal income tax
df_certified_2018['federal_income_tax'] = df_certified_2018.apply(federal_income_tax_2018,axis=1)

# print(df_certified_2018['state'].unique())
# selected_states = list(df_certified_2018['state'].unique())

# defualt state tax rate for tax free states
df_certified_2018['state_income_tax_rate'] = df_certified_2018.apply(lambda x: 0, axis=1)
df_certified_2018['state_income_tax'] = df_certified_2018.apply(lambda x: 0, axis=1)

df_certified_2018.state_income_tax_rate[df_certified_2018.state.isin(['WA','TX','AK','FL','NV'])] = df_certified_2018.apply(lambda x: 0, axis=1)
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="MI"] = df_certified_2018.apply(lambda x: 4.25, axis=1)
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="MA"] = df_certified_2018.apply(lambda x: 5.10, axis=1)
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="NC"] = df_certified_2018.apply(lambda x: 5.50, axis=1)
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="IL"] = df_certified_2018.apply(lambda x: 4.95, axis=1)
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="PA"] = df_certified_2018.apply(lambda x: 3.07, axis=1)
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="CO"] = df_certified_2018.apply(lambda x: 4.63, axis=1)
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="IN"] = df_certified_2018.apply(lambda x: 3.23, axis=1)
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="TN"] = df_certified_2018.apply(lambda x: 3.00, axis=1)

# tax brackets for selected states
# https://taxfoundation.org/
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="CA"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,8223,19495,30769,42711,53980,275738,330884,551473,1000000,np.inf], 
                                                                                labels=[1.00,2.00,3.00,4.00,8.00,9.30,10.30,11.30,12.30,13.30])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="GA"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,750,2250,2750,5250,7000,np.inf], 
                                                                                labels=[1.00,2.00,3.00,4.00,5.00,6.00])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="AZ"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,10346,25861,51721,155159, np.inf], 
                                                                     labels=[2.59,2.88,3.36,4.24,4.54])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="NJ"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,20000,35000,40000,75000,500000, np.inf], 
                                                                     labels=[1.40,1.75,3.50,5.53,6.37,8.97])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="NY"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,8500,11700,13900,21400,80650,215400,1077550, np.inf], 
                                                                     labels=[4.00,4.50,5.25,5.90,6.33,6.57,6.85,8.82])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="MD"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,1000,2000,3000,100000,125000,150000,250000, np.inf], 
                                                                     labels=[2.00,3.00,4.00,4.75,5.00,5.25,5.50,5.75])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="WI"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,11230,22470,247350, np.inf], 
                                                                     labels=[4.00,5.84,6.27,7.65])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="VA"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,3000,5000,17000, np.inf], 
                                                                     labels=[2.00,3.00,5.00,5.75])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="MN"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,25890,85060,160020, np.inf], 
                                                                     labels=[5.35,7.05,7.85,9.85])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="DC"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,10000,40000,60000,350000,1000000, np.inf], 
                                                                     labels = [4.00,6.00,6.50,8.50,8.75,8.95])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="MO"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[100,1008,2016,3024,4032,5040,6048,7056,8064,9072, np.inf], 
                                                                     labels = [1.50,2.00,2.50,3.00,3.50,4.00,4.50,5.00,5.50,5.90])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="KS"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[2500,15000,30000, np.inf], 
                                                                     labels = [3.10,5.25,5.70])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="OR"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,3450,8700,125000, np.inf], 
                                                                     labels = [5.00,7.00,9.00,9.90])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="OK"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,1000,2500,3750,4900,7200, np.inf], 
                                                                     labels = [0.50,1.00,2.00,3.00,4.00,5.00])
df_certified_2018.state_income_tax_rate[df_certified_2018.state=="HI"] = pd.cut(df_certified_2018['avg_annual_wage'], bins=[0,2400,4800,9600,14400,19200,24000,36000,48000,150000,175000,200000, np.inf], 
                                                                     labels = [1.40,3.20,5.50,6.40,6.80,7.20,7.60,7.90,8.25,9.00,10.00,11.00])

df_certified_2018['state_income_tax'] = df_certified_2018.apply(lambda x: round(x['state_income_tax_rate']*x['avg_annual_wage']/100,2), axis=1)

# https://www.irs.gov/taxtopics/tc751
df_certified_2018['social_security_tax'] = df_certified_2018.apply(lambda x: round(6.2*x['avg_annual_wage']/100,2) if x['avg_annual_wage']<128400.00 else round(6.2*128400.00/100,2), axis=1)
df_certified_2018['medicare_tax'] = df_certified_2018.apply(lambda x: round(1.45*x['avg_annual_wage']/100,2), axis=1)

df_certified_2018['all_taxes'] = df_certified_2018.apply(lambda x: x['federal_income_tax']+x['state_income_tax']+x['social_security_tax']+x['medicare_tax'], axis=1)
df_certified_2018['wage_after_taxes'] = df_certified_2018.apply(lambda x: x['avg_annual_wage']-x['all_taxes'], axis=1)


In [9]:
# ## selected cities - salary percentile plot

x_lables=selected_metros
y_labels=['P25','P50','P75','P90']

x_metro_data=[]
for x_metro in selected_metros:
    x_metro_values = df_certified_2018[df_certified_2018.metro==x_metro]['avg_annual_wage'].values
    q25, q50, q75, q90 = np.percentile(x_metro_values,[25,50,75,90])
    x_metro_data.append([round(q25,2),round(q50,2),round(q75,2),round(q90,2)])

traces = []
for ylabel_idx,y_label in enumerate(y_labels):
    ylabel_data_points = [x_metro_data[metro_idx][ylabel_idx] for metro_idx in range(len(x_metro_data))]
    traces.append(go.Bar(x=selected_metros,y=ylabel_data_points,name=y_labels[ylabel_idx]))
    
data = [traces[i] for i in range(len(traces))]

layout = go.Layout(barmode='stack', title='Gross Salary Percentile Distribution in Selected Cities (FY-2018)',yaxis=dict(title='Salary (Cumulative)'))

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_cities_percentile_salary_2018.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_selected_cities_percentile_salary_2018.html'

In [10]:
# # chart
# # selected cities - salary percentile plot  (Modified)
# # includes all information in one plot

# def hover_for_salary_percentile_plot(avg_taxes, avg_wage_after_taxes, taxes_pct):
#     result = 'Avg taxes paid by this segment ~ ' + str(convert_amount_to_human_format(avg_taxes)) + '<br>Avg wage after tax ~ ' + str(convert_amount_to_human_format(avg_wage_after_taxes)) + '<br>' + str(taxes_pct) + '% of H1B Taxes in this city' 
#     return(result)


# x_lables=selected_metros
# y_labels=['<P25 (Bottom 25%)','<P50 (Median)','>P75 (Top 25%)','>P90 (Top 10%)']

# x_metro_data=[]
# y_labels_hover_text=[]
# for x_metro in selected_metros:
#     x_metro_values = df_certified_2018[df_certified_2018.metro==x_metro]['avg_annual_wage'].values
#     q25, q50, q75, q90 = np.percentile(x_metro_values,[25,50,75,90])
    
#     x_metro_sum_alltaxes = np.nansum(df_certified_2018[df_certified_2018.metro==x_metro]['all_taxes'].values)
    
#     q25_avg_taxes = np.nanmean(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage<q25)]['all_taxes'].values)
#     q25_avg_wage_after_taxes = np.nanmean(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage<q25)]['wage_after_taxes'].values)
#     q25_taxes_pct = round(np.nansum(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage<q25)]['all_taxes'].values)/x_metro_sum_alltaxes*100,2)
    
#     q50_avg_taxes = np.nanmean(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage<q50)]['all_taxes'].values)
#     q50_avg_wage_after_taxes = np.nanmean(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage<q50)]['wage_after_taxes'].values)
#     q50_taxes_pct = round(np.nansum(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage<q50)]['all_taxes'].values)/x_metro_sum_alltaxes*100,2)
    
#     top25_avg_taxes = np.nanmean(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage>=q75)]['all_taxes'].values)
#     top25_avg_wage_after_taxes = np.nanmean(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage>=q75)]['wage_after_taxes'].values)
#     top25_taxes_pct = round(np.nansum(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage>=q75)]['all_taxes'].values)/x_metro_sum_alltaxes*100,2)
    
#     top10_avg_taxes = np.nanmean(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage>=q90)]['all_taxes'].values)
#     top10_avg_wage_after_taxes = np.nanmean(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage>=q90)]['wage_after_taxes'].values)
#     top10_taxes_pct = round(np.nansum(df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage>=q90)]['all_taxes'].values)/x_metro_sum_alltaxes*100,2)
    
#     x_metro_data.append([round(q25,2),round(q50,2),round(q75,2),round(q90,2)])
#     y_labels_hover_text.append( [hover_for_salary_percentile_plot(q25_avg_taxes,q25_avg_wage_after_taxes,q25_taxes_pct), 
#                                  hover_for_salary_percentile_plot(q50_avg_taxes,q50_avg_wage_after_taxes,q50_taxes_pct), 
#                                  hover_for_salary_percentile_plot(top25_avg_taxes,top25_avg_wage_after_taxes,top25_taxes_pct), 
#                                  hover_for_salary_percentile_plot(top10_avg_taxes,top10_avg_wage_after_taxes,top10_taxes_pct)])
    

# traces = []
# for ylabel_idx,y_label in enumerate(y_labels):
#     ylabel_data_points = [x_metro_data[metro_idx][ylabel_idx] for metro_idx in range(len(x_metro_data))]
#     ylabel_hover_text = [y_labels_hover_text[idx][ylabel_idx] for idx in range(len(y_labels_hover_text))]
#     traces.append(go.Bar(x=selected_metros,y=ylabel_data_points,text=ylabel_hover_text,name=y_labels[ylabel_idx]))
    
# data = [traces[i] for i in range(len(traces))]

# layout = go.Layout(barmode='stack', title='Gross Salary Percentile Distribution in Selected Cities (FY-2018)',yaxis=dict(title='Salary (Cumulative)'))

# fig = go.Figure(data=data, layout=layout)
# plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_cities_percentile_salary_2018.html')

In [11]:
# chart
# selected cities - outliers

x_lables=selected_metros
y_labels=['Low Income Segment <P25-1.5*IQR','Affluent Segment >P75+1.5*IQR']

x_metro_data=[]
y_labels_hover_text=[]
for x_metro in selected_metros:
    x_metro_values = df_certified_2018[df_certified_2018.metro==x_metro]['avg_annual_wage'].values
    q25, q75 = np.percentile(x_metro_values,[25,75])
    IQR = q75-q25
    
    l_outlier_wage = q25-1.5*IQR
    h_outlier_wage = q75+1.5*IQR
    
    l_outliers = list(filter(lambda x: x < l_outlier_wage ,x_metro_values))
    h_outliers = list(filter(lambda x: x > h_outlier_wage ,x_metro_values))
    x_metro_data.append([round(len(l_outliers)/len(x_metro_values)*100,2), round(len(h_outliers)/len(x_metro_values)*100,2)])
    
    y_labels_hover_text.append(
        [
            'Wage < ' + str(convert_amount_to_human_format(l_outlier_wage)),
            'Wage > ' + str(convert_amount_to_human_format(h_outlier_wage))
        ])

traces = []
for ylabel_idx,y_label in enumerate(y_labels):
    ylabel_data_points = [x_metro_data[metro_idx][ylabel_idx] for metro_idx in range(len(x_metro_data))]
    ylabel_hover_text = [y_labels_hover_text[idx][ylabel_idx] for idx in range(len(y_labels_hover_text))]
    traces.append(go.Bar(x=selected_metros,y=ylabel_data_points,text=ylabel_hover_text, name=y_labels[ylabel_idx]))
    
data = [traces[i] for i in range(len(traces))]

layout = go.Layout(barmode='stack',title='Outlier Population Proportions in Selected Cities (FY-2018)',
                   yaxis=dict(title='% of H1B certified population'), legend=dict(x=0,y=1,traceorder='normal',orientation="h"))
fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_cities_outliers_2018.html')


# def hover_for_outlier_salary_percentile_plot(avg_taxes, taxes_pct):
#     result =  'Avg taxes paid by this segment ~ ' + str(convert_amount_to_human_format(avg_taxes)) + '<br>' + str(taxes_pct) + '% of H1B Taxes in this city' 
#     return(result)

# x_lables=selected_metros
# y_labels=['Low Income Segment <P25-1.5*IQR','Affluent Segment >P75+1.5*IQR']

# x_metro_data=[]
# y_labels_hover_text=[]
# for x_metro in selected_metros:
#     x_metro_values = df_certified_2018[df_certified_2018.metro==x_metro]['avg_annual_wage'].values
    
#     q25, q75 = np.percentile(x_metro_values,[25,75])
#     IQR = q75-q25
    
#     l_outlier_wage = q25-1.5*IQR
#     h_outlier_wage = q75+1.5*IQR
    
#     l_outliers = list(filter(lambda x: x < l_outlier_wage ,x_metro_values))
#     h_outliers = list(filter(lambda x: x > h_outlier_wage ,x_metro_values))
    
#     x_metro_taxes = df_certified_2018[df_certified_2018.metro==x_metro]['all_taxes'].values
    
#     h_outliers_taxes = df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage>=h_outlier_wage)]['all_taxes'].values
#     h_outliers_avg_taxes = np.nanmean(h_outliers_taxes)
#     h_outliers_taxes_pct = round(np.nansum(h_outliers_taxes)/np.nansum(x_metro_taxes)*100, 2)
    
#     l_outliers_taxes = df_certified_2018[(df_certified_2018.metro==x_metro) & (df_certified_2018.avg_annual_wage<=l_outlier_wage)]['all_taxes'].values
#     l_outliers_avg_taxes = np.nanmean(l_outliers_taxes) if len(l_outliers_taxes)>0 else 0
#     l_outliers_taxes_pct = round(np.nansum(l_outliers_taxes)/np.nansum(x_metro_taxes)*100, 2)
    
#     x_metro_data.append([round(len(l_outliers)/len(x_metro_values)*100,2), round(len(h_outliers)/len(x_metro_values)*100,2)])
#     y_labels_hover_text.append(
#         [
#             'Wage < ' + str(convert_amount_to_human_format(l_outlier_wage)) + '<br>' + hover_for_outlier_salary_percentile_plot(l_outliers_avg_taxes, l_outliers_taxes_pct),
#             'Wage > ' + str(convert_amount_to_human_format(h_outlier_wage)) + '<br>' + hover_for_outlier_salary_percentile_plot(h_outliers_avg_taxes, h_outliers_taxes_pct)
#         ])

# traces = []
# for ylabel_idx,y_label in enumerate(y_labels):
#     ylabel_data_points = [x_metro_data[metro_idx][ylabel_idx] for metro_idx in range(len(x_metro_data))]
#     ylabel_hover_text = [y_labels_hover_text[idx][ylabel_idx] for idx in range(len(y_labels_hover_text))]
#     traces.append(go.Bar(x=selected_metros,y=ylabel_data_points,text=ylabel_hover_text, name=y_labels[ylabel_idx]))
    
# data = [traces[i] for i in range(len(traces))]

# layout = go.Layout(barmode='stack',title='Outlier Population Proportions in Selected Cities (FY-2018)',
#                    yaxis=dict(title='% of H1B certified population'), legend=dict(x=0,y=1,traceorder='normal',orientation="h"))

# fig = go.Figure(data=data, layout=layout)
# plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_cities_outliers_2018.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_selected_cities_outliers_2018.html'

In [12]:
# chart
# Gini in selected cities 2018

# x_lables=selected_metros
# y_labels=['Gini Index (Income Disparity)']

# x_metro_data=[]
# for x_metro in selected_metros:
#     x_metro_values = df_certified_2018[df_certified_2018.metro==x_metro]['avg_annual_wage'].values
#     x_metro_gini = gini(x_metro_values)
#     x_metro_data.append(round(x_metro_gini,2))

# traces = []
# ylabel_data_points = [x_metro_data[metro_idx] for metro_idx in range(len(x_metro_data))]
# traces.append(go.Bar(x=selected_metros,y=ylabel_data_points))
    
# data = [traces[i] for i in range(len(traces))]

# layout = go.Layout(barmode='stack', title='Gini Index for Selected Cities (FY-2018)',yaxis=dict(title='Salary (Cumulative)'))

# fig = go.Figure(data=data, layout=layout)
# plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_cities_gini_2018.html')

In [13]:
# chart
# Gini in selected cities, 2011-2018

df1 = df[(df.case_status=="C") & (df.metro.isin(selected_metros))]
fiscal_years = sorted(df1['fiscal_year'].unique())
traces_a = []
for x_metro in selected_metros:
    y_values = []
    for year in fiscal_years:
        df_temp = df1[(df1.fiscal_year==year) & (df1.metro==x_metro)]
        df_temp['avg_annual_wage'] = df_temp.apply(calc_emp_avg_annual_wage,axis=1)
        values = df_temp['avg_annual_wage'].values
        values = values[np.isfinite(values)]
        x_metro_gini = gini(values)
        y_values.append(round(x_metro_gini,2))
    traces_a.append(go.Scatter(x=fiscal_years,y=y_values, name=x_metro))

data = [traces_a[i] for i in range(len(traces_a))]
layout = go.Layout(title='Gini in Selected Cities (2011-2018)',yaxis=dict(title='Gini Index'), hovermode='closest')
fig = go.Figure(data,layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_cities_gini.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_selected_cities_gini.html'

In [14]:
# chart
# Median salary in selected cities, 2011-2018

df1 = df[(df.case_status=="C") & (df.metro.isin(selected_metros))]
fiscal_years = sorted(df1['fiscal_year'].unique())
traces_a = []
for x_metro in selected_metros:
    y_values = []
    for year in fiscal_years:
        df_temp = df1[(df1.fiscal_year==year) & (df1.metro==x_metro)]
        df_temp['avg_annual_wage'] = df_temp.apply(calc_emp_avg_annual_wage,axis=1)
        q50 = np.nanpercentile(df_temp['avg_annual_wage'].values,50)
        y_values.append(q50)
    traces_a.append(go.Scatter(x=fiscal_years,y=y_values, name=x_metro))

data = [traces_a[i] for i in range(len(traces_a))]
layout = go.Layout(title='Median Salary in Selected Cities (2011-2018)',yaxis=dict(title='Median salary'), hovermode='closest')
fig = go.Figure(data,layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_cities_median_salary.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_selected_cities_median_salary.html'

### Employer Analysis

In [15]:
# employer analysis

# df1 = df[(df.case_status=="C") & (df.emp_name.notnull()==True)]
df1 = df[df.emp_name.notnull()==True]
df1['emp_name_x'] = df1['emp_name']

#employer data cleansing
df1.emp_name_x[df1.emp_name.str.match('INFOSYS')] = 'INFOSYS'
df1.emp_name_x[df1.emp_name.str.match('WIPRO')] = 'WIPRO'
df1.emp_name_x[df1.emp_name.str.match('TATA')] = 'TATA (TCS)'
df1.emp_name_x[df1.emp_name.str.match('COGNIZANT')] = 'COGNIZANT'
df1.emp_name_x[df1.emp_name.str.match('CAPGEMINI')] = 'CAPGEMINI'
df1.emp_name_x[df1.emp_name.str.match('DELOITTE')] = 'DELOITTE'
df1.emp_name_x[df1.emp_name.str.match('TECH MAHINDRA')] = 'TECH MAHINDRA'
df1.emp_name_x[df1.emp_name.str.match('IBM')]='IBM'
df1.emp_name_x[df1.emp_name.str.match('L&T|(?=.*LARSEN)(?=.*TOUBRO)')] = 'L&T'
df1.emp_name_x[df1.emp_name.str.match('ACCENTURE')] = 'ACCENTURE'
df1.emp_name_x[df1.emp_name.str.match('HCL')] = 'HCL'
df1.emp_name_x[df1.emp_name.str.match('IGATE')] = 'IGATE'
df1.emp_name_x[df1.emp_name.str.match('UST GLOBAL')] = 'UST GLOBAL'
df1.emp_name_x[df1.emp_name.str.match('(?=.*SYNTEL)(?!.*SYNTELLI|SYNTELLIA)')] = 'SYNTEL'

df1.emp_name_x[df1.emp_name.str.match('PRICEWATERHOUSECOOPERS')] = 'PWC'
df1.emp_name_x[df1.emp_name.str.match('(?=.*ERNST)(?=.*YOUNG)')] = 'ERNST & YOUNG'
df1.emp_name_x[df1.emp_name.str.match('VISA U.S.A.')] = 'VISA'
df1.emp_name_x[df1.emp_name.str.match('CITIBANK|CITIGROUP|CITICORP|CITIFINANCIAL|CITIMORTGAGE|CITI VENTURES|CITI HEDGE FUND')] = 'CITIBANK'
df1.emp_name_x[df1.emp_name.str.match('DISCOVER PRODUCTS|DISCOVER GROUP|DISCOVER BANK')] = 'DISCOVER'
df1.emp_name_x[df1.emp_name.str.match('JPMORGAN')] = 'JPMORGAN CHASE'
df1.emp_name_x[df1.emp_name.str.match('(?=.*AMERICAN EXPRESS)(?!.*LOGISTICS)')] = 'AMERICAN EXPRESS'
df1.emp_name_x[df1.emp_name.str.match('WELLS FARGO')] = 'WELLS FARGO'
df1.emp_name_x[df1.emp_name.str.match('BANK OF AMERICA')] = 'BANK OF AMERICA'
df1.emp_name_x[df1.emp_name.str.match('(?=.*CAPITAL ONE)(?!.*CONSTRUCTION)')] = 'CAPITAL ONE'
df1.emp_name_x[df1.emp_name.str.match('GOLDMAN SACH')] = 'GOLDMAN SACH'
df1.emp_name_x[df1.emp_name.str.match('MORGAN STANLEY')] = 'MORGAN STANLEY'

df1.emp_name_x[df1.emp_name.str.match('AMAZON')] = 'AMAZON'
df1.emp_name_x[df1.emp_name.str.match('GOOGLE')] = 'GOOGLE'
df1.emp_name_x[df1.emp_name.str.match('FACEBOOK')] = 'FACEBOOK'
df1.emp_name_x[df1.emp_name.str.match('APPLE INC')] = 'APPLE'
df1.emp_name_x[df1.emp_name.str.match('TESLA')] = 'TESLA'
df1.emp_name_x[df1.emp_name.str.match('MICROSOFT')] = 'MICROSOFT'
df1.emp_name_x[df1.emp_name.str.match('YAHOO')] = 'YAHOO'
df1.emp_name_x[df1.emp_name.str.match('ORACLE AMERICA|ORACLE FINANCIAL|ORACLE USA')] = 'ORACLE'
df1.emp_name_x[df1.emp_name.str.match('NETFLIX')] = 'NETFLIX'
df1.emp_name_x[df1.emp_name.str.match('UBERCAB|UBER, INC|UBER TECHNOLOGIES')] = 'UBER'
df1.emp_name_x[df1.emp_name.str.match('AIRBNB')] = 'AIRBNB'
df1.emp_name_x[df1.emp_name.str.match('QUALCOMM')] = 'QUALCOMM'
df1.emp_name_x[df1.emp_name.str.match('INTEL CORPORATION|INTEL AMERICA|INTEL INC')] = 'INTEL'
df1.emp_name_x[df1.emp_name.str.match('CISCO SYSTEMS|CISCO SYSTEM|CISCO CONSUMER|CISCO FC')] = 'CISCO'

df1.emp_name_x[df1.emp_name.str.match('WAL-MART')] = 'WALMART'
df1.emp_name_x[df1.emp_name.str.match('COSTCO')] = 'COSTCO'
df1.emp_name_x[df1.emp_name.str.match('HOME DEPOT')] = 'HOME DEPOT'
df1.emp_name_x[df1.emp_name.str.match('MACYS')] = 'MACYS'
df1.emp_name_x[df1.emp_name.str.match('VERIZON')] = 'VERIZON'
df1.emp_name_x[df1.emp_name.str.match('AT&T')] = 'AT&T'

df1.emp_name_x[df1.emp_name.str.match('CHEVRON')] = 'CHEVRON'
df1.emp_name_x[df1.emp_name.str.match('EXXON MOBIL|EXXONMOBIL')] = 'EXXON MOBIL'
df1.emp_name_x[df1.emp_name.str.match('BP AMERICA')] = 'BRITISH PETROLEUM (BP)'
df1.emp_name_x[df1.emp_name.str.match('SHELL EXPLORATION|SHELL INTERNATIONAL|SHELL GLOBAL|SHELL OIL')] = 'SHELL OIL'

df1.emp_name_x[df1.emp_name.str.match('MASTECH DIGITAL')] = 'MASTECH'
df1.emp_name_x[df1.emp_name.str.match('NTT DATA')] = 'NTT DATA'
df1.emp_name_x[df1.emp_name.str.match('SALESFORCE.COM')] = 'SALESFORCE'
df1.emp_name_x[df1.emp_name.str.match('LINKEDIN')] = 'LINKEDIN'
df1.emp_name_x[df1.emp_name.str.match('VIRTUSA')] = 'VIRTUSA'
df1.emp_name_x[df1.emp_name.str.match('VMWARE')] = 'VMWARE'
df1.emp_name_x[df1.emp_name.str.match('PAYPAL')] = 'PAYPAL'
df1.emp_name_x[df1.emp_name.str.match('EBAY')] = 'EBAY'
df1.emp_name_x[df1.emp_name.str.match('HEXAWARE')] = 'HEXAWARE'
df1.emp_name_x[df1.emp_name.str.match('FUJITSU AMERICA')] = 'FUJITSU AMERICA'
df1.emp_name_x[df1.emp_name.str.match('EMC CORPORATION')] = 'EMC'
df1.emp_name_x[df1.emp_name.str.match('SATYAM COMPUTER SERVICES')] = 'SATYAM COMPUTER SERVICES'
df1.emp_name_x[df1.emp_name.str.match('PATNI AMERICAS')] = 'PATNI'

df1['avg_annual_wage'] = df1.apply(calc_emp_avg_annual_wage, axis=1)

In [16]:
# Popular Employers, who hires a lot?

# prepare data
df2 = df1[df1.case_status.isin(["C","CW"])].groupby(['fiscal_year','emp_name_x'])['case_status'].size().to_frame().reset_index()
df2.rename(columns={'case_status':'fy_company_certified_count'}, inplace=True)

fiscal_years = list(df2['fiscal_year'].unique())

emp_names = []
for year in fiscal_years:
    df_x = df2[df2.fiscal_year==year].sort_values(by=['fy_company_certified_count'],ascending=False).head(20)
    for emp in df_x['emp_name_x']:
        if(emp not in emp_names):
            emp_names.append(emp)
emp_names = list(set(emp_names))
del df2

selected_companies = ['INFOSYS','WIPRO','TATA (TCS)','COGNIZANT','CAPGEMINI','DELOITTE','IBM',
                      'CITIBANK','AMERICAN EXPRESS','BANK OF AMERICA','CAPITAL ONE','JPMORGAN CHASE','WELLS FARGO',
                      'AMAZON','GOOGLE','MICROSOFT','ORACLE','APPLE','TESLA','NETFLIX','UBER','FACEBOOK', 'AIRBNB','INTEL','CISCO',
                      'CHEVRON','EXXON MOBIL','SHELL OIL','BRITISH PETROLEUM (BP)','WALMART','COSTCO','VERIZON','AT&T']

emp_names = emp_names + selected_companies
emp_names = list(set(emp_names))
emp_names = sorted(emp_names)

df2 = df1[df1.emp_name_x.isin(emp_names)].groupby(['fiscal_year','emp_name_x'])['case_status'].size().to_frame().reset_index()
df2.rename(columns={'case_status':'fy_company_count'}, inplace=True)
df3 = df1[df1.case_status.isin(["C","CW"])].groupby(['fiscal_year','emp_name_x'])['case_status'].size().to_frame().reset_index()
df3.rename(columns={'case_status':'fy_company_certified_count'}, inplace=True)
df2 = pd.merge(df2,df3,on=['fiscal_year','emp_name_x'])
del df3
df2['approval_rate'] = df2.apply(lambda x: round(x['fy_company_certified_count']/x['fy_company_count']*100,2), axis=1)

df3 = df1[(df1.case_status=="C") & (df1.emp_name_x.isin(emp_names))]
df3 = df3.groupby(['fiscal_year','emp_name_x'])['avg_annual_wage'].median().to_frame().reset_index()
df3.rename(columns={'avg_annual_wage':'avg_annual_median_wage'}, inplace=True)
df2 = pd.merge(df2,df3,on=['fiscal_year','emp_name_x'])
del df3

df2['category'] = df2['emp_name_x']
df2.category[df2.emp_name_x.str.match('ACCENTURE|CAPGEMINI|COGNIZANT|DELOITTE|FUJITSU|HCL|IBM|IGATE|INFOSYS|L&T|PATNI|SATYAM|SYNTEL|TATA|TECH MAHINDRA|UST GLOBAL|WIPRO')] = 'CONSULTING'
df2.category[df2.emp_name_x.str.match('AMAZON|APPLE|CISCO|FACEBOOK|GOOGLE|INTEL|MICROSOFT|NETFLIX|ORACLE|QUALCOMM|TESLA|UBER|AIRBNB')] = 'TECH'
df2.category[df2.emp_name_x.str.match('AMERICAN EXPRESS|BANK OF AMERICA|CAPITAL ONE|CITIBANK|ERNST & YOUNG|JPMORGAN CHASE|PWC|WELLS FARGO')] = 'FINANCIAL'
df2.category[df2.emp_name_x.str.match('AT&T|VERIZON')] = 'TELECOM'
df2.category[df2.emp_name_x.str.match('BRITISH|CHEVRON|SHELL|EXXON')] = 'OIL SECTOR'
df2.category[df2.emp_name_x.str.match('WALMART|COSTCO')] = 'RETAIL'

df2['text'] = df2.apply(lambda x: x['emp_name_x'] + '<br>No of Certified LCA(s): ' + str(x['fy_company_certified_count']) , axis=1)
# chart
categories = list(df2['category'].unique())

figure = {
    'data': [],
    'layout': {},
    'frames': []
}

figure['layout']['xaxis'] = {'range': [80, 100], 'title': 'Approval Rate'}
figure['layout']['yaxis'] = {'range': [20000, np.nanmax(df2['avg_annual_median_wage'])] , 'title': 'Median Salary'}
figure['layout']['title'] = 'Popular Employers (2011-2018)<br><span style="font-size:x-small;width:50%;">NOTE: This is an interactive chart, please hover over individual bubbles for more detailed information. Also you may selectively enable or disable legend items to isolate individual category or more.</span>'

figure['layout']['hovermode'] = 'closest'
figure['layout']['legend'] = { 'font' : {'family':'sans-serif','size':12} }
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': '1952',
    'plotlycommand': 'animate',
    'values': fiscal_years,
    'visible': True
}
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

# make data
year = fiscal_years[0]
for category in categories:
    dataset_by_year = df2[df2.fiscal_year == year]
    dataset_by_year_and_cat = dataset_by_year[dataset_by_year.category == category]

    data_dict = {
        'x': list(dataset_by_year_and_cat['approval_rate']),
        'y': list(dataset_by_year_and_cat['avg_annual_median_wage']),
        'mode': 'markers',
        'text': list(dataset_by_year_and_cat['text']),
        'marker': {
            'sizemode': 'area',
            'sizeref': 1,
            'size': list(dataset_by_year_and_cat['fy_company_certified_count'])
        },
        'name': category
    }
    figure['data'].append(data_dict)
    
# make frames
for year in fiscal_years:
    frame = {'data': [], 'name': str(year)}
    for category in categories:
        dataset_by_year = df2[df2.fiscal_year == year]
        dataset_by_year_and_cat = dataset_by_year[dataset_by_year.category == category]

        data_dict = {
            'x': list(dataset_by_year_and_cat['approval_rate']),
            'y': list(dataset_by_year_and_cat['avg_annual_median_wage']),
            'mode': 'markers',
            'text': list(dataset_by_year_and_cat['text']),
            'marker': {
                'sizemode': 'area',
                'sizeref': 1,
                'size': list(dataset_by_year_and_cat['fy_company_certified_count'])
            },
            'name': category
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': str(year),
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

    
figure['layout']['sliders'] = [sliders_dict]

plotly.offline.plot(figure, include_plotlyjs=include_plotlyjs, filename='oflc_popular_employers.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_popular_employers.html'

In [17]:
selected_companies = ['INFOSYS','WIPRO','TATA (TCS)','COGNIZANT','CAPGEMINI','DELOITTE','IBM',
                      'CITIBANK','AMERICAN EXPRESS','BANK OF AMERICA','CAPITAL ONE','JPMORGAN CHASE','WELLS FARGO',
                      'AMAZON','GOOGLE','MICROSOFT','ORACLE','APPLE','TESLA','NETFLIX','UBER','FACEBOOK', 'AIRBNB','INTEL','CISCO',
                      'CHEVRON','EXXON MOBIL','SHELL OIL','BRITISH PETROLEUM (BP)','WALMART','COSTCO','VERIZON','AT&T'
                     ]

selected_companies = sorted(selected_companies)

In [18]:
# chart
# selected companies - salary percentile plot

oflc_2018_selected_companies = df1[(df1.fiscal_year==2018) & (df1.case_status=="C") & (df1.emp_name_x.isin(selected_companies))]
oflc_2018_selected_companies['avg_annual_wage'] = oflc_2018_selected_companies.apply(calc_emp_avg_annual_wage, axis=1)

x_lables=selected_companies
y_labels=['P25','P50','P75','P90']

x_emp_data=[]
for x_emp in selected_companies:
    x_emp_values = oflc_2018_selected_companies[oflc_2018_selected_companies.emp_name_x==x_emp]['avg_annual_wage'].values
    q25, q50, q75, q90 = np.percentile(x_emp_values,[25,50,75,90])
    x_emp_data.append([round(q25,2),round(q50,2),round(q75,2),round(q90,2)])

traces = []
for ylabel_idx,y_label in enumerate(y_labels):
    ylabel_data_points = [x_emp_data[emp_idx][ylabel_idx] for emp_idx in range(len(x_emp_data))]
    traces.append(go.Bar(x=selected_companies,y=ylabel_data_points,name=y_labels[ylabel_idx]))
    
data = [traces[i] for i in range(len(traces))]

layout = go.Layout(barmode='stack', title='Percentile-Salary Distribution in Selected Companies (FY-2018)',yaxis=dict(title='Salary (Cumulative)'))

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_companies_percentile_salary_2018.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_selected_companies_percentile_salary_2018.html'

In [19]:
# chart
# selected companies - outliers
x_lables=selected_companies
y_labels=['Low Income Segment <P25-1.5*IQR','Affluent Segment >P75+1.5*IQR']

x_emp_data=[]
y_labels_hover_text=[]
for x_emp in selected_companies:
    x_metro_values = oflc_2018_selected_companies[oflc_2018_selected_companies.emp_name_x==x_emp]['avg_annual_wage'].values
    
    q25, q75 = np.percentile(x_metro_values,[25,75])
    IQR = q75-q25
    
    l_outlier_wage = q25-1.5*IQR
    h_outlier_wage = q75+1.5*IQR
    
    l_outliers = list(filter(lambda x: x < l_outlier_wage, x_metro_values))
    h_outliers = list(filter(lambda x: x > h_outlier_wage, x_metro_values))
    
    x_emp_data.append([round(len(l_outliers)/len(x_metro_values)*100,2), round(len(h_outliers)/len(x_metro_values)*100,2)])
    y_labels_hover_text.append(
        [
            'Wage < ' + str(convert_amount_to_human_format(l_outlier_wage)),
            'Wage > ' + str(convert_amount_to_human_format(h_outlier_wage))
        ])

traces = []
for ylabel_idx,y_label in enumerate(y_labels):
    ylabel_data_points = [x_emp_data[emp_idx][ylabel_idx] for emp_idx in range(len(x_emp_data))]
    ylabel_hover_text = [y_labels_hover_text[idx][ylabel_idx] for idx in range(len(y_labels_hover_text))]
    traces.append(go.Bar(x=selected_companies,y=ylabel_data_points,text=ylabel_hover_text, name=y_labels[ylabel_idx]))
    
data = [traces[i] for i in range(len(traces))]

layout = go.Layout(barmode='stack',title='Outlier Population Proportions in Selected Companies (FY-2018)',
                   yaxis=dict(title='% of H1B certified population'), legend=dict(x=0,y=1,traceorder='normal',orientation="h"))

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_companies_outliers_2018.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_selected_companies_outliers_2018.html'

In [20]:
# chart
# Median salary in selected companies, 2011-2018

dfx = df1[(df1.case_status=="C") & (df1.emp_name_x.isin(selected_companies))]
fiscal_years = sorted(dfx['fiscal_year'].unique())
traces_a = []
for x_emp in selected_companies:
    y_values = []
    for year in fiscal_years:
        df_temp = dfx[(dfx.fiscal_year==year) & (dfx.emp_name_x==x_emp)]
        if(len(df_temp)>0):
            df_temp['avg_annual_wage'] = df_temp.apply(calc_emp_avg_annual_wage,axis=1)
            q50, q90 = np.percentile(df_temp['avg_annual_wage'].values,[50,90])
            y_values.append(q50)
        else:
            y_values.append(np.nan)
    traces_a.append(go.Scatter(x=fiscal_years,y=y_values, name=x_emp))

data = [traces_a[i] for i in range(len(traces_a))]
layout = go.Layout(title='Median Salary in Selected Companies (2011-2018)',yaxis=dict(title='Median salary'), hovermode='closest')
fig = go.Figure(data,layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_companies_median_salary.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_selected_companies_median_salary.html'

In [21]:
# chart
# Gini in selected companies, 2011-2018

dfx = df1[(df1.case_status=="C") & (df1.emp_name_x.isin(selected_companies))]
fiscal_years = sorted(dfx['fiscal_year'].unique())
traces_a = []
for x_emp in selected_companies:
    y_values = []
    for year in fiscal_years:
        df_temp = dfx[(dfx.fiscal_year==year) & (dfx.emp_name_x==x_emp)]
        if(len(df_temp)>0):
            df_temp['avg_annual_wage'] = df_temp.apply(calc_emp_avg_annual_wage,axis=1)
            values = df_temp['avg_annual_wage'].values
            values = values[np.isfinite(values)]
            x_emp_gini = gini(values)
            y_values.append(round(x_emp_gini,2))
        else:
            y_values.append(np.nan)
    traces_a.append(go.Scatter(x=fiscal_years,y=y_values, name=x_emp))

data = [traces_a[i] for i in range(len(traces_a))]
layout = go.Layout(title='Gini in Selected Companies (2011-2018)',yaxis=dict(title='Gini Index'), hovermode='closest')
fig = go.Figure(data,layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_companies_gini.html')


'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_selected_companies_gini.html'

In [22]:
# chart
# Application Volume and Approval rate plots of selected companies, 2011-2018

df2 = df1.groupby(['fiscal_year','emp_name_x'])['case_status'].size().to_frame().reset_index()
df2.rename(columns={'case_status':'fy_company_count'}, inplace=True)

df3 = df1[df1.case_status.isin(["C","CW"])].groupby(['fiscal_year','emp_name_x'])['case_status'].size().to_frame().reset_index()
df3.rename(columns={'case_status':'fy_company_certified_count'}, inplace=True)

df2 = pd.merge(df2,df3,on=['fiscal_year','emp_name_x'])

df2['approval_rate'] = df2.apply(lambda x: round(x['fy_company_certified_count']/x['fy_company_count']*100,2), axis=1)
df2 = df2.drop(['fy_company_certified_count'],axis=1)

del df3

fiscal_years = sorted(df2['fiscal_year'].unique())
traces_a = []
for x_emp in selected_companies:
    y_values = []
    for year in fiscal_years:
        df_temp = df2[(df2.fiscal_year==year) & (df2.emp_name_x==x_emp)]
        if(len(df_temp)>0):
            y_values.append(df_temp['fy_company_count'].values[0])
        else:
            y_values.append(np.nan)
    traces_a.append(go.Scatter(x=fiscal_years,y=y_values, name=x_emp))

data = [traces_a[i] for i in range(len(traces_a))]
layout = go.Layout(title='Application Volume of Selected Companies (2011-2018)',yaxis=dict(title='Application Volume'), hovermode='closest')
fig = go.Figure(data,layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_companies_application_volume.html')

traces_a = []
for x_emp in selected_companies:
    y_values = []
    for year in fiscal_years:
        df_temp = df2[(df2.fiscal_year==year) & (df2.emp_name_x==x_emp)]
        if(len(df_temp)>0):
            y_values.append(df_temp['approval_rate'].values[0])
        else:
            y_values.append(np.nan)
    traces_a.append(go.Scatter(x=fiscal_years,y=y_values, name=x_emp))

data = [traces_a[i] for i in range(len(traces_a))]
layout = go.Layout(title='Approval Rates of Selected Companies (2011-2018)',yaxis=dict(title='Approval Percentage'), hovermode='closest')
fig = go.Figure(data,layout)
plotly.offline.plot(fig, include_plotlyjs=include_plotlyjs, filename='oflc_selected_companies_approval_rates.html')

'file:///Users/genie/dev/projects/github/h1b_data_analysis/out/oflc_selected_companies_approval_rates.html'